# Data preparation
This playbook does the following things:
- Calculates simple change in colum <column_name>_sp_change
- Calculates percentage change in column <column_name>_pct_change
- calculates 5,20,50 simple/exponential moving averages in columns 5_sma,5_ema etc.

In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
# retrieve data from s3
from gw2ml.data.s3_sync import download_folder_from_s3
import pandas as pd
import os

datapath_raw = os.path.join(os.getenv('DATA_PATH'), 'workdir_lusis', 'raw')
os.makedirs(datapath_raw, exist_ok=True)

datapath_calculated = os.path.join(os.getenv('DATA_PATH'), 'workdir_lusis', 'calculated')
os.makedirs(datapath_calculated, exist_ok=True)

download_folder_from_s3(s3_folder_prefix='datasources/gw2/raw/1764510842', local_folder=datapath_raw)

Source: ost-s3/datasources/gw2/raw/1764510842
Target: /models/data/workdir_lusis/raw

Found 280 files to download

⊘ +1 Agony Infusion.csv (already exists, ETag matches)
⊘ Aetherkey Piece.csv (already exists, ETag matches)
⊘ Ancient Bone.csv (already exists, ETag matches)
⊘ Ancient Wood Log.csv (already exists, ETag matches)
⊘ Ancient Wood Plank.csv (already exists, ETag matches)
⊘ Antique Summoning Stone.csv (already exists, ETag matches)
⊘ Armored Scale.csv (already exists, ETag matches)
⊘ Asparagus Spear.csv (already exists, ETag matches)
⊘ Auric Sliver.csv (already exists, ETag matches)
⊘ Bag of Alchemical Materials.csv (already exists, ETag matches)
⊘ Bag of Coffee Beans.csv (already exists, ETag matches)
⊘ Bag of Flour.csv (already exists, ETag matches)
⊘ Bag of Starch.csv (already exists, ETag matches)
⊘ Bag of Stolen Goods.csv (already exists, ETag matches)
⊘ Bag of Sugar.csv (already exists, ETag matches)
⊘ Bay Leaf.csv (already exists, ETag matches)
⊘ Beet.csv (already exists

In [3]:
import glob

# Load all CSV files into a list of dictionaries containing filename and dataframe
csv_files = glob.glob(os.path.join(datapath_raw, '*.csv'))
data_list = [
    {
        'filename': os.path.basename(filepath),
        'name': os.path.splitext(os.path.basename(filepath))[0],  # filename without extension
        'df': pd.read_csv(filepath, delimiter=';')
    }
    for filepath in csv_files
]

print(f"Loaded {len(data_list)} CSV files:")
for item in data_list:
    print(f"  - {item['filename']}: {item['df'].shape[0]} rows, {item['df'].shape[1]} columns")

Loaded 280 CSV files:
  - Silver Ore.csv: 30906 rows, 10 columns
  - Rotted Titan Amber.csv: 30951 rows, 10 columns
  - Pouch of Orange Pigment.csv: 30929 rows, 10 columns
  - Spring Roll.csv: 30922 rows, 10 columns
  - Strawberry Ghost.csv: 30917 rows, 10 columns
  - Copper Ore.csv: 30906 rows, 10 columns
  - Trick-or-Treat Bag.csv: 30964 rows, 10 columns
  - Walnut.csv: 30832 rows, 10 columns
  - Omnomberry.csv: 30881 rows, 10 columns
  - Orange.csv: 30832 rows, 10 columns
  - Carrot.csv: 30829 rows, 10 columns
  - Pile of Ascalonian Herbs.csv: 30832 rows, 10 columns
  - Turnip.csv: 30832 rows, 10 columns
  - Kralkachocolate Bar.csv: 30904 rows, 10 columns
  - Lump of Coal.csv: 30906 rows, 10 columns
  - Slab of Poultry Meat.csv: 30848 rows, 10 columns
  - Flawless Snowflake.csv: 30888 rows, 10 columns
  - Bay Leaf.csv: 30832 rows, 10 columns
  - Superior Trebuchet Blueprints.csv: 30937 rows, 10 columns
  - Yam.csv: 30832 rows, 10 columns
  - Ugly Wool Sweater.csv: 30935 rows, 10 col

In [4]:

def calculate_simple_change(series):
    """Calculate simple change (current - previous)"""
    return series.diff()

def calculate_pct_change(series):
    """Calculate percentage change"""
    return series.pct_change() * 100

def calculate_sma(series, window_size):
    """Calculate Simple Moving Average"""
    return series.rolling(window=window_size, min_periods=1).mean()

def calculate_ema(series, window_size):
    """Calculate Exponential Moving Average"""
    return series.ewm(span=window_size, adjust=False).mean()

In [6]:
# Process each dataset
from tqdm import tqdm

window_sizes = [5, 20, 50]

for item in tqdm(data_list, desc="Processing datasets"):
    print(f"Processing {item['name']}")
    df = item['df'].copy()

    # Identify numeric columns (excluding any date/time columns)
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    # Remove item_id from numeric columns if present
    if 'item_id' in numeric_cols:
        numeric_cols.remove('item_id')

    for col in numeric_cols:
        # Calculate simple change
        df[f'{col}_sp_change'] = calculate_simple_change(df[col])

        # Calculate percentage change
        df[f'{col}_pct_change'] = calculate_pct_change(df[col])

        # Calculate SMAs and EMAs for different window sizes
        for window in window_sizes:
            df[f'{col}_{window}_sma'] = calculate_sma(df[col], window)
            df[f'{col}_{window}_ema'] = calculate_ema(df[col], window)

    # Save the processed dataframe
    output_path = os.path.join(datapath_calculated, f"{item['name']}_calculated.csv")
    df.to_csv(output_path, index=False, sep=';')

    print(f"  - Saved to {output_path}")
    print(f"  - Original columns: {len(item['df'].columns)}, New columns: {len(df.columns)}")

print("\nProcessing complete!")

Processing datasets:   0%|          | 0/280 [00:00<?, ?it/s]

Processing Silver Ore


Processing datasets:   0%|          | 1/280 [00:00<02:50,  1.64it/s]

  - Saved to /models/data/workdir_lusis/calculated/Silver Ore_calculated.csv
  - Original columns: 10, New columns: 50
Processing Rotted Titan Amber


Processing datasets:   1%|          | 2/280 [00:01<02:43,  1.70it/s]

...

  - Saved to /models/data/workdir_lusis/calculated/Small Wintersday Gift_calculated.csv
  - Original columns: 10, New columns: 50
Processing Bowl of Coleslaw


Processing datasets: 100%|█████████▉| 279/280 [02:31<00:00,  1.90it/s]

  - Saved to /models/data/workdir_lusis/calculated/Bowl of Coleslaw_calculated.csv
  - Original columns: 10, New columns: 50
Processing Pile of Vile Essence


Processing datasets: 100%|██████████| 280/280 [02:32<00:00,  1.84it/s]

  - Saved to /models/data/workdir_lusis/calculated/Pile of Vile Essence_calculated.csv
  - Original columns: 10, New columns: 50

Processing complete!
